In [ ]:
import os
import json
import torch
from torchvision import transforms
from PIL import Image
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

from models import SimpleViT


In [ ]:
MODEL_PATH = "vit.cth"
LABEL_MAP_PATH = "vit_map.json"
TEST_DIR = "./Test"
OUTPUT_CSV = "jawaban.csv"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


In [ ]:
with open(LABEL_MAP_PATH, "r") as f:
    class_to_idx = json.load(f)

idx_to_class = {v: k for k, v in class_to_idx.items()}
num_classes = len(idx_to_class)

idx_to_class


In [ ]:
model = SimpleViT(num_classes=num_classes, pretrained=False)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.to(device)
model.eval()

print("Model loaded successfully!")


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


In [ ]:
def predict_image(img_path):
    img = Image.open(img_path).convert("RGB")
    x = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        logits = model(x)
        pred_index = logits.argmax(dim=1).item()

    return pred_index


In [ ]:
y_true = []
y_pred = []
filenames = []

for label_name in os.listdir(TEST_DIR):
    class_folder = os.path.join(TEST_DIR, label_name)
    if not os.path.isdir(class_folder):
        continue

    for file in os.listdir(class_folder):
        path = os.path.join(class_folder, file)
        if file.lower().endswith((".jpg", ".jpeg", ".png")):

            true_label = class_to_idx[label_name]
            pred_label = predict_image(path)

            y_true.append(true_label)
            y_pred.append(pred_label)
            filenames.append(f"{label_name}/{file}")


In [ ]:
df = pd.DataFrame({
    "filename": filenames,
    "true_label_index": y_true,
    "true_label_name": [idx_to_class[i] for i in y_true],
    "pred_label_index": y_pred,
    "pred_label_name": [idx_to_class[i] for i in y_pred],
})

df.to_csv(OUTPUT_CSV, index=False)
df


In [ ]:
print("Accuracy:", accuracy_score(y_true, y_pred))

print("\nClassification Report:")
print(classification_report(
    y_true,
    y_pred,
    target_names=[idx_to_class[i] for i in range(num_classes)],
    zero_division=0
))


In [ ]:
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(
    cm,
    annot=False,
    cmap="Blues",
    xticklabels=[idx_to_class[i] for i in range(num_classes)],
    yticklabels=[idx_to_class[i] for i in range(num_classes)],
)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()
